In [1]:
import numpy as np 
from Objects.sudoku import Sudoku
from Objects.population import Population
from Operators.fitness import fitness
from Operators.conflicts import box_conflicts, row_conflicts, col_conflicts
from Algorithms.search import Sim_annealing, Hill_climbing
import pandas as pd

import time
import tqdm

In [2]:
test_board = np.array([[5, 4, 3, 6, 7, 9, 1, 2, 8],
       [0, 7, 0, 3, 2, 5, 6, 0, 4],
       [0, 2, 6, 1, 4, 8, 5, 3, 7],
       [1, 9, 0, 2, 8, 0, 0, 4, 6],
       [4, 0, 2, 9, 1, 7, 8, 5, 3],
       [7, 0, 0, 4, 5, 6, 9, 1, 2],
       [3, 1, 9, 7, 6, 2, 0, 8, 0],
       [0, 8, 0, 0, 3, 1, 2, 0, 9],
       [0, 0, 7, 0, 9, 0, 0, 6, 0]])

#### Testing genetic algorithms

In [3]:
population = Population(size=100, initial_board=test_board)

In [4]:
population.get_distances(normalize=True)

array([0.6462585 , 0.57823129, 0.46258503, 0.44217687, 0.29931973,
       0.04761905, 0.76870748, 0.61904762, 0.68027211, 0.76870748,
       0.39455782, 0.29931973, 0.26530612, 0.        , 0.44217687,
       0.2244898 , 0.44217687, 0.3537415 , 0.57823129, 0.39455782,
       0.4829932 , 0.78911565, 0.53061224, 0.23129252, 0.60544218,
       0.34013605, 0.61904762, 0.41496599, 0.6122449 , 0.55102041,
       0.40816327, 0.27210884, 0.55782313, 0.50340136, 0.44217687,
       0.52380952, 0.68027211, 0.67346939, 1.        , 0.49659864,
       0.47619048, 0.57823129, 0.49659864, 0.66666667, 0.44897959,
       0.51020408, 0.44897959, 0.6462585 , 0.66666667, 0.56462585,
       0.63265306, 0.52380952, 0.34013605, 0.37414966, 0.4829932 ,
       0.31292517, 0.27210884, 0.41496599, 0.46938776, 0.53061224,
       0.54421769, 0.54421769, 0.40136054, 0.46938776, 0.66666667,
       0.46258503, 0.59863946, 0.33333333, 0.55782313, 0.68707483,
       0.36054422, 0.46258503, 0.46938776, 0.30612245, 0.42176

In [21]:
population.evolve(
    gens = 10000, 
    xo_prob = 1,
    mut_prob = 1, 
    select_type='roulette',
    xo='single_point',
    mutation='change',   # Change instead of swap
    elite_size=20,
    swap_number=2
)

Best individual of gen #1: 22. Mean fitness: 34.36
Best individual of gen #2: 22. Mean fitness: 33.74
Best individual of gen #3: 22. Mean fitness: 34.0
Best individual of gen #4: 22. Mean fitness: 34.265
Best individual of gen #5: 22. Mean fitness: 33.565
Best individual of gen #6: 22. Mean fitness: 33.82
Best individual of gen #7: 22. Mean fitness: 33.385
Best individual of gen #8: 22. Mean fitness: 34.23
Best individual of gen #9: 22. Mean fitness: 34.05
Best individual of gen #10: 22. Mean fitness: 34.605
Best individual of gen #11: 22. Mean fitness: 33.68
Best individual of gen #12: 22. Mean fitness: 34.205
Best individual of gen #13: 22. Mean fitness: 33.725
Best individual of gen #14: 22. Mean fitness: 34.135
Best individual of gen #15: 22. Mean fitness: 34.085
Best individual of gen #16: 22. Mean fitness: 33.4
Best individual of gen #17: 22. Mean fitness: 33.375
Best individual of gen #18: 22. Mean fitness: 32.865
Best individual of gen #19: 22. Mean fitness: 33.735


KeyboardInterrupt: 

In [2]:
# Lets do grid search to find the best parameters for the hill climbing algorithm
hill_climbing_args={'num_neighbours': np.arange(1, 10, 1), 
                    'swap_number': np.arange(1, 10, 1)}

num_iterations = 20
num_combinations = 1

means = {}
stds = {}
unable_args = []

for combination in tqdm.tqdm(range(num_combinations)):
    unable = False
    # Get the random parameters
    # arg_dictionnary = {'num_neighbours': np.random.choice(hill_climbing_args['num_neighbours']),
    #                     'swap_number': np.random.choice(hill_climbing_args['swap_number']), 
    #                     'max_iterations': 100000,
    #                     'plateau_threshold': 100
    #                     }

    arg_dictionnary = {'num_neighbours': 10*combination+1,
                        'swap_number': 1, 
                        'max_iterations': 100000,
                        'plateau_threshold': 1000,
                        'verbose': 2
                        }
    
    temp_results = []
    for iteration in range(num_iterations):
        if unable:
            break
        start = time.time()
        try:
            Sudoku(hill_climbing_args=arg_dictionnary)
        except:
            unable = True
            continue

        finish = time.time()
        temp_results.append(finish-start)
    
    if unable:
        print('Unable to solve the puzzle with arguments: ', arg_dictionnary)
        unable_args.append(arg_dictionnary)
        continue

    means[arg_dictionnary['num_neighbours'], arg_dictionnary['swap_number']] = np.mean(temp_results)
    stds[arg_dictionnary['num_neighbours'], arg_dictionnary['swap_number']] = np.std(temp_results)

100%|██████████| 1/1 [00:00<?, ?it/s]

Unable to solve the puzzle with arguments:  {'num_neighbours': 1, 'swap_number': 1, 'max_iterations': 100000, 'plateau_threshold': 1000, 'verbose': 2}


#### Initialization of the sudoku arguments

In [3]:
sudoku = Sudoku(initial_board=np.zeros((9,9), dtype=int), fill_board='random')
sudoku.display()


 9  5  1  |  9  2  2  |  6  5  7 
 6  1  3  |  1  3  1  |  2  7  2 
 3  8  8  |  1  1  3  |  6  4  9 
----------|-----------|----------
 4  2  6  |  8  4  5  |  4  8  3 
 6  8  1  |  3  3  4  |  3  4  6 
 8  9  7  |  5  9  8  |  7  7  3 
----------|-----------|----------
 8  7  4  |  7  7  1  |  2  9  6 
 6  6  4  |  4  5  2  |  5  9  5 
 9  9  5  |  2  8  2  |  7  5  1 


In [17]:
sudoku = Sudoku()

In [15]:
sim_annealing_algo = Sim_annealing(sudoku)
sim_annealing_algo.run(L=100, c=20, alpha=0.95)

[[3 6 2 9 1 4 7 5 8]
 [9 1 4 5 7 8 3 2 6]
 [5 7 8 3 2 6 9 1 4]
 [8 9 1 2 3 5 4 7 6]
 [2 4 5 1 6 7 8 9 3]
 [7 3 6 8 4 9 1 2 5]
 [6 5 7 4 9 1 2 8 9]
 [1 8 9 6 8 2 5 3 7]
 [4 2 3 7 5 3 6 4 1]]

In [ ]:
# Lets do grid search to find the best parameters for the hill climbing algorithm
hill_climbing_args={'num_neighbours': np.arange(1, 10, 1), 
                    'swap_number': np.arange(1, 10, 1)}

num_iterations = 20
num_combinations = 1

means = {}
stds = {}
unable_args = []

for combination in tqdm.tqdm(range(num_combinations)):
    unable = False
    # Get the random parameters
    # arg_dictionnary = {'num_neighbours': np.random.choice(hill_climbing_args['num_neighbours']),
    #                     'swap_number': np.random.choice(hill_climbing_args['swap_number']), 
    #                     'max_iterations': 100000,
    #                     'plateau_threshold': 100
    #                     }

    arg_dictionnary = {'num_neighbours': 10*combination+1,
                        'swap_number': 1, 
                        'max_iterations': 100000,
                        'plateau_threshold': 1000,
                        'verbose': 2
                        }
    
    temp_results = []
    for iteration in range(num_iterations):
        if unable:
            break
        start = time.time()
        try:
            Sudoku(hill_climbing_args=arg_dictionnary)
        except:
            unable = True
            continue

        finish = time.time()
        temp_results.append(finish-start)
    
    if unable:
        print('Unable to solve the puzzle with arguments: ', arg_dictionnary)
        unable_args.append(arg_dictionnary)
        continue

    means[arg_dictionnary['num_neighbours'], arg_dictionnary['swap_number']] = np.mean(temp_results)
    stds[arg_dictionnary['num_neighbours'], arg_dictionnary['swap_number']] = np.std(temp_results)

100%|██████████| 1/1 [00:00<?, ?it/s]

Unable to solve the puzzle with arguments:  {'num_neighbours': 1, 'swap_number': 1, 'max_iterations': 100000, 'plateau_threshold': 1000, 'verbose': 2}


In [211]:
fitness_time = {}
num_combs = 100
num_iters = 20

for i in tqdm.tqdm(range(num_combs)):
    alpha = np.random.uniform(0.5, 0.99)
    c = np.random.randint(1, 50)
    L = np.random.randint(1, 50)

    temp_times = []
    temp_fitness = []

    for j in range(num_iters):
        start = time.time()
        fitness = sim_annealing(sudoku, verbose=0, alpha=alpha, c=c, L=L).fitness
        finish = time.time()
        temp_times.append(finish-start)
        temp_fitness.append(fitness)

    mean_time = np.mean(temp_times)
    mean_fitness = np.mean(temp_fitness)
    fitness_time[alpha, c, L, mean_fitness] = mean_time


df = pd.DataFrame.from_dict(fitness_time, orient='index', columns=['time'])
# Sepparate the index into columns
df['alpha'] = df.index.map(lambda x: x[0])
df['c'] = df.index.map(lambda x: x[1])
df['L'] = df.index.map(lambda x: x[2])
df['fitness'] = df.index.map(lambda x: x[3])
# Drop the index
df = df.reset_index().drop(columns='index')

df[df['fitness'] < 10]

100%|██████████| 100/100 [17:47<00:00, 10.68s/it]


In [41]:
sudoku.fitness

82

In [50]:
filled_sudoku = hill_climbing(sudoku, max_iterations=100, plateau_threshold=100, verbose=2, num_neighbours=1, swap_number=1)
filled_sudoku = sim_annealing(filled_sudoku, verbose=1, alpha=0.978727, c=4, L=33)
filled_sudoku.fitness

Reached max iterations, returned [[9 5 6 8 8 2 1 4 6]
 [5 3 5 1 4 5 8 7 2]
 [3 1 8 6 2 7 7 2 3]
 [8 7 5 7 9 1 3 2 6]
 [7 2 9 5 3 9 4 8 1]
 [7 6 1 4 9 2 3 5 8]
 [2 4 6 1 6 8 9 4 7]
 [1 9 3 5 4 1 6 3 7]
 [4 4 6 9 9 3 2 8 5]]
SA found with fitness 18


2

In [53]:
filled_sudoku = sim_annealing(sudoku, verbose=1, alpha=0.985, c=10, L=5)
filled_sudoku = hill_climbing(filled_sudoku, verbose=1, num_neighbours=1, swap_number=2, max_iterations=100000, plateau_threshold=1000)

SA found with fitness 25
Reached max iterations, returned [[2 1 7 8 3 4 5 9 6]
 [5 9 4 6 2 7 1 3 8]
 [3 8 6 9 5 1 2 7 4]
 [9 3 8 4 1 6 2 5 7]
 [4 7 5 2 8 9 3 4 1]
 [1 6 2 3 7 5 8 6 9]
 [6 2 9 5 4 8 7 1 3]
 [7 4 3 1 9 2 6 8 5]
 [8 5 1 7 6 3 4 2 9]]


In [198]:
filled_sudoku = hill_climbing(sudoku, verbose=1, num_neighbours=10, swap_number=1, max_iterations=100000, plateau_threshold=1000)

In [4]:
filled_sudoku.display()


 1  6  5  |  8  2  3  |  7  4  9 
 2  7  9  |  6  1  4  |  5  3  8 
 3  4  8  |  9  5  7  |  6  2  1 
----------|-----------|----------
 3  9  6  |  1  4  5  |  8  7  2 
 5  4  2  |  3  8  2  |  9  1  6 
 3  7  1  |  2  6  9  |  8  5  4 
----------|-----------|----------
 8  5  7  |  3  9  2  |  1  6  4 
 6  3  9  |  5  1  8  |  4  5  7 
 7  1  4  |  9  7  6  |  3  2  8 


In [5]:
# Iterate over all the positions to get where the conflict is 
for i in range(9):
    print(f'{i+1}th Box conflicts: {box_conflicts(filled_sudoku.board, i)}')
    print(f'{i+1}th Row conflicts: {row_conflicts(filled_sudoku.board, i)}')
    print(f'{i+1}th Col conflicts: {col_conflicts(filled_sudoku.board, i)}')

1th Box conflicts: 0
1th Row conflicts: 0
1th Col conflicts: 2
2th Box conflicts: 0
2th Row conflicts: 0
2th Col conflicts: 2
3th Box conflicts: 0
3th Row conflicts: 0
3th Col conflicts: 1
4th Box conflicts: 1
4th Row conflicts: 0
4th Col conflicts: 2
5th Box conflicts: 1
5th Row conflicts: 1
5th Col conflicts: 1
6th Box conflicts: 1
6th Row conflicts: 0
6th Col conflicts: 1
7th Box conflicts: 1
7th Row conflicts: 0
7th Col conflicts: 1
8th Box conflicts: 1
8th Row conflicts: 1
8th Col conflicts: 2
9th Box conflicts: 1
9th Row conflicts: 1
9th Col conflicts: 2


In [3]:
new_individual = Sudoku(hill_climbing_args={'max_iterations' : 100000, 
                                            'verbose': 3, 
                                            'num_neighbours': 100,
                                            'plateau_threshold': 10000})

Iteration 0 : Found a better solution with fitness: 74
Iteration 1 : Found a better solution with fitness: 71
Iteration 2 : Found a better solution with fitness: 67
Iteration 3 : Found a better solution with fitness: 63
Iteration 4 : Found a better solution with fitness: 60
Iteration 5 : Found a better solution with fitness: 57
Iteration 6 : Found a better solution with fitness: 54
Iteration 7 : Found a better solution with fitness: 52
Iteration 8 : Found a better solution with fitness: 50
Iteration 9 : Found a better solution with fitness: 48
Iteration 10 : Found a better solution with fitness: 46
Iteration 11 : Found a better solution with fitness: 44
Iteration 12 : Found a better solution with fitness: 43
Iteration 13 : Found a better solution with fitness: 42
Iteration 14 : Found a better solution with fitness: 39
Iteration 15 : Found a better solution with fitness: 37
Iteration 16 : Found a better solution with fitness: 36
Iteration 17 : Found a better solution with fitness: 35
It

AttributeError: 'Sudoku' object has no attribute 'initial_board'